In [10]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

import numpy as np
import pandas as pd

import os

In [2]:
from scipy import stats
from sklearn.discriminant_analysis import StandardScaler


def janelaDeslizante(vetor, janelaEntrada, janelaSaida):
    """
    Calcula a matriz de conhecimento

    Args:
        vetor: Array com os valores da série temporal.
        janelaEntrada: int com o tamanho da janela de entrada.
        JanelaSaida: int com o tamanho da janela de saída.

    Returns:
        X: matriz com as entradas do modelo tem tamanho (numero de amostras X tamanho da janela entrada)
        y: matriz com as saídas do modelo tem tamanho (numero de amostras X tamanho da janela de saida)
    """
    #### YEO johnson
    transformacao, lambda_param = stats.yeojohnson(vetor)
    X_y = transformacao
    
    ## Padronização
    scaler = StandardScaler()
    f = np.array(X_y).reshape(-1,1)
    X_p = scaler.fit_transform(f)
    
    X_p = np.array(X_p).flatten()
    
    num_amostras = len(X_p) - janelaEntrada - janelaSaida + 1
    X = np.zeros((num_amostras, janelaEntrada))
    y = np.zeros((num_amostras, janelaSaida))
    
    for i in range(num_amostras):
        X[i] = X_p[i : i + janelaEntrada]
        y[i] = X_p[i + janelaEntrada : i + janelaEntrada + janelaSaida]

    return X, y

In [3]:
def dividirConjuntoDados(X,y, tamanhoTeste=0.2):
  """
    Divide o conjunto de dados em treino e teste dado a porcentagem para o teste, sem realizar o embaralhamento
    
    Args:
        X: matriz com as entradas do modelo tem tamanho (numero de amostras X tamanho da janela entrada)
        Y: matriz com as saídas do modelo tem tamanho (numero de amostras X tamanho da janela de saida)
        tamanhoTeste: float porcentagem referente ao tamanho do teste

    Returns:
        y_pred: matriz com as previsoes realizadas pelo modelo ajustado
    """
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tamanhoTeste, random_state=42, shuffle=False)
  
  return X_train,y_train,X_test,y_test

In [4]:
def ajustarModelo (X_train, y_train):
  import tensorflow as tf
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import LSTM, Dense, Dropout
  """
    Ajusta o modelo de LSTM
    
    Args:
        X_train: matriz com as entradas do modelo tem tamanho (numero de amostras X tamanho da janela entrada)
        y_train: matriz com as saídas do modelo tem tamanho (numero de amostras X tamanho da janela de saida)
        X_test: matriz com as entradas para teste do modelo tem tamanho (numero de amostras X tamanho da janela entrada)

    Returns:
        y_pred: matriz com as previsoes realizadas pelo modelo ajustado
        Retorna o modelo para ser salvo e posteriormente realizar as metricas de avaliação
    """
    
  model = Sequential()
  model.add(LSTM(units=64, input_shape=(X.shape[1], 1),return_sequences=True))
  
  for _ in range(1):
      model.add(LSTM(units=64, return_sequences=True))
      model.add(Dropout(0.2))
  
  model.add(LSTM(units=64))
  model.add(Dropout(0.2))  # Regularização Dropout
  model.add(Dense(units=1))  # Saída única para previsão de valor futuro

  # Compilar o modelo
  model.compile(optimizer='adam', loss='mean_squared_error')

  # Treinar o modelo
  model.fit(X_train, y_train, epochs=500, batch_size=1, verbose=2)

  # # Fazer previsões
  # predictions = model.predict(X_test)

  # # Fazer previsões
  # y_pred = model.predict(X_test)

  
  return model


In [5]:
dados_2019 = pd.read_excel('../codigos/data/2019.xlsx')

In [6]:
janeiro_2019 = dados_2019[:21]
fevereiro_2019 = dados_2019[21:41]
marco_2019 = dados_2019[41:60]
abril_2019 = dados_2019[60:81]
maio_2019 = dados_2019[81:103]
junho_2019 = dados_2019[103:122]
julho_2019 = dados_2019[122:144]
agosto_2019 = dados_2019[144:166]
setembro_2019 = dados_2019[166:187]
outubro_2019 = dados_2019[187:210]
novembro_2019 = dados_2019[210:229]
dezembro_2019 = dados_2019[229:248]

incrementos = [[], janeiro_2019, fevereiro_2019, marco_2019, abril_2019, maio_2019, junho_2019, julho_2019, agosto_2019, setembro_2019, outubro_2019, novembro_2019, dezembro_2019]
meses = ['1. Janeiro', '2. Fevereiro', '3. Março', '4. Abril', '5. Maio', '6. Junho', '7. Julho', '8. Agosto', '9. Setembro', '10. Outubro', '11. Novembro', '12. Dezembro']

In [7]:
janelas = [ '1. jan 2018 - dez 2018   -> jan 2019',
            '2. jan 2018 - jan 2019   -> fev 2019',
            '3. jan 2018 - fev 2019   -> mar 2019',
            '4. jan 2018 - mar 2019   -> abr 2019',
            '5. jan 2018 - abr 2019   -> mai 2019',
            '6. jan 2018 - mai 2019   -> jun 2019',
            '7. jan 2018 - jun 2019   -> jul 2019',
            '8. jan 2018 - jul 2019   -> ago 2019',
            '9. jan 2018 - ago 2019   -> set 2019',
            '10. jan 2018 - set 2019   -> out 2019',
            '11. jan 2018 - out 2019   -> nov 2019',
            '12. jan 2018 - nov 2019   -> dez 2019']

In [8]:
## MODELO INDIVIDUAL com periodo deslizante


for i, janela in enumerate(janelas):
  print(i)
  
  if i == 0:
    dados = pd.read_excel('../codigos/data/2018.xlsx')
    ativos = dados.columns[1:]
  else:
    dados = np.concatenate((dados,incrementos[i]), axis=0)
  dados = pd.DataFrame(dados, columns=ativos.insert(0, 'date'))
  
  if i <= 7:
    print(i)
    continue
  
  for ativo in ativos:
    print('ajuste ', i, janela)
    print(dados)    
        
    if os.path.exists(f'models/individuais/2018/deslizante/{janela}/{ativo}.h5'):
      print("já existe")
      print(f'models/individuais/2018/deslizante/{janela}/{ativo}.h5')
    else:
        
      print(dados[ativo])
      # break
      X,y = janelaDeslizante(np.array(dados[ativo], float), 22,1)

      X_train,y_train = X, y

      model = ajustarModelo(X_train, y_train)
      
      print(f'models/individuais/2018/deslizante/{janela}/{ativo}.h5')
      model.save(f'models/individuais/2018/deslizante/{janela}/{ativo}.h5')

0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
ajuste  8 9. jan 2018 - ago 2019   -> set 2019
          date      ABEV3      B3SA3      BBAS3      BBDC3      BBDC4  \
0          NaT  21.690001   7.636666      32.93   18.37306  19.693554   
1          NaT      21.74   7.766666  33.349998  18.441362  19.773239   
2          NaT  21.620001   7.816666  33.669998  18.777178   20.09767   
3          NaT  21.700001       7.93  33.669998  18.782869  20.211506   
4          NaT      21.66   7.966666      33.75  18.782869  20.205814   
..         ...        ...        ...        ...        ...        ...   
407 2019-08-26      18.09       14.1  44.740002  21.059353  23.771601   
408 2019-08-27      18.17  14.333333  44.380001  21.322313  24.072126   
409 2019-08-28      18.15  14.543333  44.009998  21.202103  23.929377   
410 2019-08-29  18.549999  14.763333  45.080002  21.878286  24.380165   
411 2019-08-30  18.719999  14.933333  46.240002  22.764837  24.755821   

         BEEF3      BRAP4      BRFS3      

In [9]:
dados

,date,ABEV3,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BRAP4,BRFS3,BRKM5,...,SBSP3,TAEE11,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VIVT3,WEGE3,YDUQ3
0,NaT,21.690001,7.636666,32.93,18.37306,19.693554,10.70808,26.223991,37.27,43.860001,...,34.599998,21.459999,13.2,10.030941,38.485001,9.6,41.720001,41.400002,9.623076,33.869999
1,NaT,21.74,7.766666,33.349998,18.441362,19.773239,10.629416,26.471889,37.0,43.490002,...,33.98,21.49,13.21,10.243869,38.105,9.92,41.470001,40.200001,9.476923,33.900002
2,NaT,21.620001,7.816666,33.669998,18.777178,20.09767,10.590085,27.224436,37.400002,44.150002,...,33.32,21.15,13.15,10.180656,38.549999,10.48,41.639999,40.639999,9.384615,33.759998
3,NaT,21.700001,7.93,33.669998,18.782869,20.211506,10.70808,27.879593,39.240002,45.220001,...,33.700001,21.25,13.26,10.400239,38.630001,10.03,42.290001,40.740002,9.423076,33.299999
4,NaT,21.66,7.966666,33.75,18.782869,20.205814,10.344261,28.366533,39.700001,45.299999,...,34.389999,21.15,13.08,10.393585,38.900002,10.14,43.23,41.169998,9.5,33.419998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,2019-11-25,18.200001,15.923333,46.599998,23.688955,25.274229,13.244981,30.066401,36.110001,27.85,...,54.369999,27.860001,13.8,21.813334,22.17,8.39,50.860001,45.400002,14.55,37.66
471,2019-11-26,18.030001,15.5,46.0,22.945154,24.733282,13.618633,30.827799,36.799999,27.9,...,54.259998,27.870001,13.62,21.686666,21.93,8.48,51.220001,44.380001,14.315,38.830002
472,2019-11-27,17.959999,15.816666,47.34,23.268219,25.146505,13.805459,30.818947,36.619999,27.76,...,54.650002,27.959999,13.65,21.886665,21.799999,8.4,50.580002,45.650002,14.525,40.650002
473,2019-11-28,18.01,16.049999,47.599998,23.24568,24.951164,14.582261,30.278885,36.200001,27.700001,...,56.25,28.129999,13.92,21.823334,21.9,8.43,50.369999,46.0,14.715,42.599998


In [ ]:
## MODELO POR GRUPOS
classificacao = pd.read_excel('../codigos/resultados/2019/classes.xlsx')
dados = pd.read_excel('../codigos/data/2018.xlsx')


classes = [0,1,2,3,4,5,6,7]

for classe in classes:
  filtro = classificacao.query(f'classes=={classe}')
  ativos = filtro['ativos']


  ## CONTROI A MATRIZ DE CONHECIMENTO PARA OS ATIVOS DO CLUSTER
  for i,ativo in enumerate(ativos):
    if i == 0 :
      X,y = janelaDeslizante(dados[ativo], 22,1)
      # X_train,y_train, X_test, y_test = dividirConjuntoDados(X,y)
      X_train,y_train = X, y
      
    else:
      X_temp,y_temp = janelaDeslizante(dados[ativo], 22,1)
      # X_train_temp, y_train_temp, X_test_temp, y_test_temp = dividirConjuntoDados(X_temp,y_temp)
      X_train_temp, y_train_temp = X_temp,y_temp
      
      X_train = np.concatenate((X_train,X_train_temp), axis=0)
      y_train = np.concatenate((y_train,y_train_temp), axis=0)
      
      # X_test = np.concatenate((X_test,X_test_temp), axis=0)
      # y_test = np.concatenate((y_test,y_test_temp), axis=0)
      
    print(ativo, i)
 
 
  ## MODELAGEM

  model = ajustarModelo(X_train, y_train)

  model.save(f'models/agrupados/2018/fixo/modeloAgrupadoClasse_{classe}.h5')

In [ ]:
### MODELO COM A MEDIA
classificacao = pd.read_excel('../codigos/resultados/2019/classes.xlsx')
dados = pd.read_excel('../codigos/data/2018.xlsx')
classes = [0,1,2,3,4,5,6,7]

for classe in classes:
  filtro = classificacao.query(f'classes=={classe}')
  ativos = filtro['ativos']

  dados['media'] = dados[ativos].T.mean()

  X,y = janelaDeslizante(dados['media'], 22,1)

  # X_train,y_train, X_test, y_test = dividirConjuntoDados(X,y)
  X_train,y_train = X, y

  model = ajustarModelo(X_train, y_train)

  model.save(f'models/media/2018/fixo/classe_{classe}.h5')

#### Resultados

In [ ]:
resultados = {}
resultados['ABEV3'] = [ "modelo individual", "modelo Ajustado com dados do cluster", "modelo Com a media"]

In [9]:
# Calcular o MAPE
def calcular_mape(y_true, y_pred):
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mape

In [ ]:
################################################################
### RESULTADOS INDIVIDUAIS

train = []
test = []

## MODELO INDIVIDUAL
dados = pd.read_excel('../codigos/data/2019.xlsx')
ativos = dados.columns[1:]

for ativo in ativos:
  X,y = janelaDeslizante(dados[ativo], 22,1)
  X_train,y_train, X_test, y_test = dividirConjuntoDados(X,y)
  modelo = tf.keras.models.load_model(f'models/individuais/{ativo}.h5')
  y_pred_train = modelo.predict(X_train)
  train.append(mean_absolute_percentage_error(y_train, y_pred_train))
  y_pred_test = modelo.predict(X_test)
  test.append(mean_absolute_percentage_error(y_test, y_pred_test))

In [100]:
len(test)

108

In [ ]:
################################################################
### RESULTADOS INDIVIDUAIS com 2018 fixo prevendo os meses de 2019


dados = pd.read_excel('../codigos/data/2018.xlsx')
ativos = dados.columns[1:]

for incremento, mes in zip(incrementos[1:], meses):
  test = []
  
  dados = pd.DataFrame(np.concatenate((dados[-22:], incremento)), columns=ativos.insert(0, 'date'))

  for ativo in ativos:
    dados_float = np.array(dados[ativo], dtype=float)
    X,y = janelaDeslizante(dados_float, 22,1)

    modelo = tf.keras.models.load_model(f'models/individuais/2018/fixo/{ativo}.h5')
    # y_pred_train = modelo.predict(X_train)
    # train.append(mean_absolute_percentage_error(y_train, y_pred_train))
    y_pred_test = modelo.predict(X)
    test.append(mean_absolute_percentage_error(y, y_pred_test))
    
  resultado = pd.DataFrame()
  # resultado["Modelo Individual Treino"] = train
  resultado['ativos']= ativos
  resultado["Modelo Individual Teste"] = test
  resultado.to_excel(f'Resultados_2018 1ª abordagem/{mes}/resultados.xlsx', index=False)

In [19]:
incremento

,Date,ABEV3,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BRAP4,BRFS3,BRKM5,...,SBSP3,TAEE11,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VIVT3,WEGE3,YDUQ3
0,2019-01-02,16.150000,9.263333,48.599998,22.395441,25.288004,4.995137,27.525454,22.379999,48.310001,...,34.369999,24.209999,12.15,9.531889,27.170000,9.76,51.090000,41.910000,8.990,24.850000
1,2019-01-03,16.330000,9.666666,48.799999,22.852491,25.532181,5.113132,26.471889,21.490000,48.599998,...,37.020000,24.280001,11.92,9.751472,26.665001,9.55,49.000000,41.400002,9.195,24.830000
2,2019-01-04,16.549999,9.316666,48.799999,22.351616,25.331829,5.309791,28.260292,21.450001,48.060001,...,37.860001,25.240000,11.98,10.020960,26.485001,10.11,52.189999,41.090000,9.300,25.209999
3,2019-01-07,16.480000,9.266666,48.480000,22.539444,25.632357,5.280293,28.481628,21.590000,47.720001,...,37.000000,24.820000,12.37,9.804704,25.674999,10.13,51.910000,41.200001,9.155,24.709999
4,2019-01-08,16.150000,9.293333,47.930000,22.789883,25.788879,5.221295,28.278000,22.930000,47.689999,...,37.139999,24.799999,12.84,9.405462,25.250000,9.77,52.410000,40.299999,9.080,24.940001
5,2019-01-09,16.379999,9.336666,47.799999,23.265715,26.233408,5.309791,29.039396,23.549999,48.009998,...,37.599998,24.799999,12.99,9.768107,26.059999,10.09,53.689999,40.720001,9.180,26.709999
6,2019-01-10,16.850000,9.423333,48.500000,23.334585,26.152016,5.349123,28.950863,23.450001,47.509998,...,37.830002,25.000000,12.50,9.947765,26.735001,10.33,53.099998,40.000000,9.250,27.100000
7,2019-01-11,17.290001,9.623333,48.700001,23.240671,25.982969,5.309791,28.853474,22.950001,48.180000,...,39.490002,25.490000,12.46,10.044249,26.594999,10.18,52.380001,40.450001,9.345,27.370001
8,2019-01-14,17.540001,9.836666,49.830002,23.265715,26.289757,5.349123,28.959717,22.670000,48.279999,...,41.599998,25.520000,12.46,10.337026,26.785000,9.87,52.599998,40.099998,9.450,28.600000
9,2019-01-15,17.700001,9.740000,49.250000,23.353369,26.239670,5.368789,28.694113,22.430000,48.330002,...,41.599998,25.150000,12.48,10.653092,26.549999,9.74,52.349998,40.360001,9.265,29.000000


In [94]:
resultado = pd.DataFrame()
# resultado["Modelo Individual Treino"] = train
resultado['ativos']= ativos
resultado["Modelo Individual Teste"] = test
resultado.to_excel('Resultados_2018 1ª abordagem/1. Janeiro/resultados.xlsx', index=False)

In [ ]:
## MODELO POR GRUPOS

classificacao = pd.read_excel('../codigos/resultados/2019/classes.xlsx')

train = []
test = []

dados = pd.read_excel('../codigos/data/2019.xlsx')
ativos = dados.columns[1:]

for ativo in ativos:
    X,y = janelaDeslizante(dados[ativo], 22,1)
    X_train,y_train, X_test, y_test = dividirConjuntoDados(X,y)
    
    classe = classificacao.query("ativos=='"+ ativo +"'")['classes'].values[0]

    modelo = tf.keras.models.load_model(f'models/agrupados/modeloAgrupadoClasse_{classe}.h5')
    y_pred_train = modelo.predict(X_train)
    train.append(mean_absolute_percentage_error(y_train, y_pred_train))
    y_pred_test = modelo.predict(X_test)
    test.append(mean_absolute_percentage_error(y_test, y_pred_test))

In [ ]:
resultado = pd.read_excel('resultados.xlsx')
resultado["Modelo por grupos Treino"] = train
resultado["Modelo por grupos Teste"] = test
resultado.to_excel('resultados.xlsx', index=False)

In [59]:
### MODELO COM A MÉDIA

train = []
test = []

dados = pd.read_excel('../codigos/data/2019.xlsx')
ativos = dados.columns[1:]

for ativo in ativos:

  X,y = janelaDeslizante(dados[ativo], 22,1)

  X_train,y_train, X_test, y_test = dividirConjuntoDados(X,y)
  
  classe = classificacao.query("ativos=='"+ ativo +"'")['classes'].values[0]

  modelo = tf.keras.models.load_model(f'models/media/classe_{classe}.h5')
  y_pred_train = modelo.predict(X_train)
  train.append(mean_absolute_percentage_error(y_train, y_pred_train))
  y_pred_test = modelo.predict(X_test)
  test.append(mean_absolute_percentage_error(y_test, y_pred_test))

In [ ]:
resultado = pd.read_excel('resultados.xlsx')
resultado["Modelo com a Média Treino"] = train
resultado["Modelo com a Média Teste"] = test
resultado.to_excel('resultados.xlsx', index=False)

In [ ]:
## ajustar os 3 tipos de modelos

## primeira abordagem
## 2018 treino    jan -> dez   prever 2019 inteiro (um modelo prevendo todos os meses)  



## segundo modelo  -> adicionar o mes ao treinamento  (ao todo serao 13 modelos)
## 2019 teste       jan 


## no final serao 13 pastas (uma para cada mes e uma com o total do ano) -> cada pasta tara os resultados de cada clusters





In [28]:
## Criar pastas para os meses
import os

# Lista de nomes dos meses
meses = ['1. Janeiro', '2. Fevereiro', '3. Março', '4. Abril', '5. Maio', '6. Junho', '7. Julho', '8. Agosto', '9. Setembro', '10. Outubro', '11. Novembro', '12. Dezembro']
meses = [ '1. jan 2018 - dez 2018   -> jan 2019',
          '2. jan 2018 - jan 2019   -> fev 2019',
          '3. jan 2018 - fev 2019   -> mar 2019',
          '4. jan 2018 - mar 2019   -> abr 2019',
          '5. jan 2018 - abr 2019   -> mai 2019',
          '6. jan 2018 - mai 2019   -> jun 2019',
          '7. jan 2018 - jun 2019   -> jul 2019',
          '8. jan 2018 - jul 2019   -> ago 2019',
          '9. jan 2018 - ago 2019   -> set 2019',
          '10. jan 2018 - set 2019   -> out 2019',
          '11. jan 2018 - out 2019   -> nov 2019',
          '12. jan 2018 - nov 2019   -> dez 2019']
# Diretório onde as pastas serão criadas
diretorio_base = 'models/agrupados/2018/deslizante'

# Verifica se o diretório base existe, senão cria
if not os.path.exists(diretorio_base):
    os.mkdir(diretorio_base)

# Cria uma pasta para cada mês
for mes in meses:
    nome_pasta = os.path.join(diretorio_base, mes)
    os.mkdir(nome_pasta)
    print(f"Pasta '{nome_pasta}' criada.")


Pasta 'models/agrupados/2018/deslizante/1. jan 2018 - dez 2018   -> jan 2019' criada.
Pasta 'models/agrupados/2018/deslizante/2. jan 2018 - jan 2019   -> fev 2019' criada.
Pasta 'models/agrupados/2018/deslizante/3. jan 2018 - fev 2019   -> mar 2019' criada.
Pasta 'models/agrupados/2018/deslizante/4. jan 2018 - mar 2019   -> abr 2019' criada.
Pasta 'models/agrupados/2018/deslizante/5. jan 2018 - abr 2019   -> mai 2019' criada.
Pasta 'models/agrupados/2018/deslizante/6. jan 2018 - mai 2019   -> jun 2019' criada.
Pasta 'models/agrupados/2018/deslizante/7. jan 2018 - jun 2019   -> jul 2019' criada.
Pasta 'models/agrupados/2018/deslizante/8. jan 2018 - jul 2019   -> ago 2019' criada.
Pasta 'models/agrupados/2018/deslizante/9. jan 2018 - ago 2019   -> set 2019' criada.
Pasta 'models/agrupados/2018/deslizante/10. jan 2018 - set 2019   -> out 2019' criada.
Pasta 'models/agrupados/2018/deslizante/11. jan 2018 - out 2019   -> nov 2019' criada.
Pasta 'models/agrupados/2018/deslizante/12. jan 2018